In [167]:
import pyautogui
import time
from pyautogui import locateOnScreen, typewrite, click, locateCenterOnScreen, locateAllOnScreen, locate

In [168]:
def refresh():
    """ 
    Refreshes flea market page. 

    Based on 1920x1080. Second sleep timer can be adjusted, but too fast will
    cause it to miss items and captchas. Do not change first sleep timer. 
    """
    flealoc = (1247, 1064)
    traderloc = (1108, 1064)
    click(traderloc)
    time.sleep(0.02) 
    click(flealoc)
    time.sleep(0.3) 


def purchase():
    """  
    Actually buys the item.

    Click locations based on 1920x1080
    The "y" keytype attempts to buy full stacks of items
    """
    click(1755, 177)
    # pyautogui.click(1148, 485) for "buy all" not needed for grenade case
    typewrite("y")


def run_captcha():
    """ 
    Checks for and completes captcha. 

    If "Security Check" is on the screen, it looks for the saved text names for items.
    Once it finds which item the captcha wants it searches for images of the item
    and confirms afterwards. 

    Returns
    -------
    bool
        True if no captcha, or if captcha successfully completed.
        False if failed (Usually because unknown captcha) saves image of screen for
        debugging.
    """
    captcha = bool(locateOnScreen("images/securitycheck.PNG", grayscale=True, confidence=0.85))
    terms = ["GunpowderEagle", "InsulatingTape", "Bolts", "AI-2medkit",
        "Screwdriver", "GraphicsCard", "BeefStew", "Wrench", "Lion",
        "Majaica", "Propane", "Sugar", "GasAnal", "RR", "Grizzly", "Vaseline",
        "Drill", "TP", "100ml", "Pliers", "Crowbar", "GoldenStar", "SparkPlug",
        "CarBattery", "Strike", "Analgin", "Zippo", "Teapot", "Moonshine",
        "Aseptic", "HotRod", "Rooster", "GPhoneX", "Alyonka", "TPlug", "Fuel",
        "Condensed", "Water", "Horse", "Sodium", "Morphine", "Tea", "Salewa", "Paper",
        "Xeno", "Goldchain", "Splint"]
    while captcha:
        print("security check")
        for term in terms:
            if locateOnScreen("images/" + term + "Text.PNG", grayscale=True, confidence=0.9):
                print(term)
                locations = locateAllOnScreen("images/" + term + ".PNG", grayscale=True, confidence=0.97)
                for loc in locations:
                    click(loc)
                    time.sleep(0.05)
                print("Confirming")
                click(locateCenterOnScreen("images/Confirm.PNG", grayscale=True, confidence=0.8))
                time.sleep(1)
                # click(locateCenterOnScreen("FleaMarket.PNG", grayscale=True, confidence=0.8))
                # time.sleep(1)
                return True
        unk = pyautogui.screenshot(imageFilename="images/UNK.PNG")
        print("Unknown Captcha", unk)
        return False
    return True


def sell_screen(item):
    """ 
    Sells current screen by finding all and looping through. 

    For Pox, it sells just under 1mil so cash is only two slots

    Parameters
    ----------
    item : string
        Base file name for stash version of the item. 
    """
    print("item", item)
    sellItem = "images/sell" + item
    with pyautogui.hold("ctrl"):
        for thingy in list(locateAllOnScreen(sellItem, grayscale=True, confidence=0.99))[:6]:
            click(thingy)
    time.sleep(0.5)
    click(locateOnScreen("images/Deal.PNG", grayscale=True, confidence=0.8))


def sell(item):
    """ 
    Sells all items.

    Sells current screen, then scrolls down and repeats

    Parameters
    ----------
    item : string
        Base file name for item including PNG extension. 
    """
    time.sleep(0.4)
    click(locateCenterOnScreen("images/Traders.PNG", grayscale=True, confidence=0.8))
    time.sleep(0.4)
    click(locateCenterOnScreen("images/Therapist.PNG", grayscale=True, confidence=0.8))
    time.sleep(0.4)
    click(locateCenterOnScreen("images/Sell.PNG", grayscale=True, confidence=0.8))
    time.sleep(0.4)
    
    for x in range(8):
        sell_screen(item)
        pyautogui.moveTo(1600, 300)
        for x in range(6):
            pyautogui.scroll(-30, x=1600, y=300)
    time.sleep(1)
    # click(locateOnScreen("FleaMarket.PNG", grayscale=True, confidence=0.8))

def make_space(item):
    """ 
    Makes space *Surprised pika face*

    Tries to move two of the item in to character inventory

    Parameters
    ----------
    item : string
        Base file name item including PNG extension. 
    """
    stashItem = "stash" + item
    time.sleep(0.5)
    click(locateCenterOnScreen("images/Character.PNG", grayscale=True, confidence=0.9))
    time.sleep(0.5)
    with pyautogui.hold("ctrl"):
        for x in range(1):
            click(locateCenterOnScreen(stashItem, grayscale=True, confidence=0.9))
    time.sleep(0.5)


def check_error(item):
    """
    Checks for errors, including no money and no stash space.

    Clicks "OK" unless no money or no space in stash

    Parameters
    ----------
    item : string
        Base file name for stash version of the item. 
    """
    errorTarget = pyautogui.screenshot(region=(700, 200, 500, 700))
    if locate(
        "images/error.PNG", errorTarget, grayscale=True, confidence=0.85) or locate(
            "images/CriticalError.PNG", errorTarget, grayscale=True, confidence=0.8
        ):
        print("error")
        if locateOnScreen("images/NoSpace.PNG", grayscale=True, confidence=0.9) or locateOnScreen(
            "images/NoSpace2.PNG", grayscale=True, confidence=0.9
            ):
            click(locateCenterOnScreen("images/OK.PNG", confidence=0.9))
            # make_space(item) not used with grenade case
            sell(item)
        click(locateCenterOnScreen("images/OK.PNG", confidence=0.9))
        time.sleep(0.25)
    if locate("images/NotEnoughMoney.PNG", errorTarget, grayscale=True, confidence=0.8):
        click(locateCenterOnScreen("images/OK.PNG", confidence=0.9))
        sell(item)


def check_price():
    """
    Not used.
    """
    target = pyautogui.screenshot(region = (1355, 155, 40, 35), imageFilename = "images/target.png")
    if locate("images/12.png", target, grayscale=True, confidence=0.95) or locate("images/13.png", target, grayscale=True, confidence=0.95):
        return False
    else:
        return True

In [175]:
list(locateAllOnScreen("images/Sugar.PNG", grayscale=True, confidence=0.95))

[Box(left=1062, top=360, width=64, height=64),
 Box(left=1188, top=486, width=64, height=64),
 Box(left=1062, top=612, width=64, height=64)]

In [169]:
def main(item):
    """
    Runs the main bot loop. 
    
    Loops endlessly, stopped if cursor in corner. First,
    it checks for an error on the screen and then a captcha. If no error or captcha
    it looks for the purchase button on the screen, and buys if it is and refreshes
    if it is not

    Parameters
    ----------
    item : string
        Base file name for image of item to buy

        Currently Implemented
        ---------------------
        1. "Pox.PNG"
        2. "Adrenaline.PNG"
        3. "GrenadeCase.PNG"
    
    IMPORTANT
    ---------
    Price needs to be filtered IN GAME not through the bot
    """
    time.sleep(2)
    while True:
        check_error(item) # sell() and make_room() handled here
        if run_captcha():
            # time.sleep(0.05) # Page loading delay, there is also a delay inside the refresh function 
            if locate("images/Purchase.PNG", pyautogui.screenshot(region=(1700, 155, 150, 50)),
                grayscale=True, confidence=0.9
            ):
                purchase()
            refresh()   
        else:
            break     

In [170]:
main("GrenadeCase.PNG")

security check
Pliers
Confirming
security check
Condensed
Confirming
security check
Zippo


FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.

In [ ]:
sell("GrenadeCase.PNG")

item GrenadeCase.PNG
item GrenadeCase.PNG
item GrenadeCase.PNG
item GrenadeCase.PNG
item GrenadeCase.PNG
item GrenadeCase.PNG
item GrenadeCase.PNG
item GrenadeCase.PNG
